# Day 13: Knights of the Dinner Table

[*Advent of Code 2015 day 13*](https://adventofcode.com/2015/day/13) and [*solution megathread*](https://www.reddit.com/3wm0oy)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/13/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F13%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

I didn't entirely feel like going at this - but considering I saw in the solution megathread that brute force pretty much was the only option, perhaps I can do it as well as anyone?

In [5]:
testdata = """Alice would gain 54 happiness units by sitting next to Bob.
Alice would lose 79 happiness units by sitting next to Carol.
Alice would lose 2 happiness units by sitting next to David.
Bob would gain 83 happiness units by sitting next to Alice.
Bob would lose 7 happiness units by sitting next to Carol.
Bob would lose 63 happiness units by sitting next to David.
Carol would lose 62 happiness units by sitting next to Alice.
Carol would gain 60 happiness units by sitting next to Bob.
Carol would gain 55 happiness units by sitting next to David.
David would gain 46 happiness units by sitting next to Alice.
David would lose 7 happiness units by sitting next to Bob.
David would gain 41 happiness units by sitting next to Carol."""

inputdata = downloaded['input']

In [6]:
print(f'{inputdata[:200]}...')

Alice would gain 2 happiness units by sitting next to Bob.
Alice would gain 26 happiness units by sitting next to Carol.
Alice would lose 82 happiness units by sitting next to David.
Alice would lose ...


In [7]:
import re


def mangle_preference(pref):
    return (pref['from'],
            pref['to'],
            int(pref['value']) * (-1 if pref['sign'] == 'lose' else 1))


def parse_data(data: str):
    output = []
    pattern = re.compile(
        r'^(?P<from>\w+) would (?P<sign>gain|lose) (?P<value>\d+) ' +
        r'happiness units by sitting next to (?P<to>\w+)\.$')
    for line in data.splitlines():
        matches = pattern.match(line)
        output.append(
            mangle_preference(
                matches.groupdict()))
    return output

In [8]:
parse_data(testdata)

[('Alice', 'Bob', 54),
 ('Alice', 'Carol', -79),
 ('Alice', 'David', -2),
 ('Bob', 'Alice', 83),
 ('Bob', 'Carol', -7),
 ('Bob', 'David', -63),
 ('Carol', 'Alice', -62),
 ('Carol', 'Bob', 60),
 ('Carol', 'David', 55),
 ('David', 'Alice', 46),
 ('David', 'Bob', -7),
 ('David', 'Carol', 41)]

In [9]:
def pivot_preferences(prefs):
    output = {}
    for f, t, v in prefs:
        if f in output.keys():
            output[f][t] = v
        else:
            output[f] = {t: v}
    return output

In [10]:
prefs = pivot_preferences(parse_data(testdata))
print(prefs)

{'Alice': {'Bob': 54, 'Carol': -79, 'David': -2}, 'Bob': {'Alice': 83, 'Carol': -7, 'David': -63}, 'Carol': {'Alice': -62, 'Bob': 60, 'David': 55}, 'David': {'Alice': 46, 'Bob': -7, 'Carol': 41}}


Around here, I'm reminded both numpy and itertools probably are great to learn, but I'll be fine...? _(No, I wasn't...)_

In [11]:
def evaluate_seating(seating, prefs):
    happiness = 0
    for i, s in enumerate(seating):
        if i == 0:
            right = seating[-1]
        else:
            right = seating[i - 1]

        if i == len(seating) - 1:
            left = seating[0]
        else:
            left = seating[i + 1]

        for n in left, right:
            happiness += prefs[s][n]
    return happiness

In [12]:
evaluate_seating(list(prefs.keys()), prefs)

330

In [13]:
from typing import Tuple


def optimal_seating(
        assigned_seats,
        remaining_seats,
        prefs,
        debug=False) -> Tuple[list, int]:
    if debug:
        print(f'{assigned_seats=}, {remaining_seats=}')
    if len(remaining_seats) == 1:
        assigned_seats_n = assigned_seats + [remaining_seats[0]]
        happiness = evaluate_seating(assigned_seats_n, prefs)
        if debug:
            print(f'{assigned_seats_n=} evaluate to {happiness=}')
        return (
            assigned_seats_n,
            happiness)
    elif len(remaining_seats) == 2:
        pick_next = optimal_seating(
            assigned_seats + [remaining_seats[1]],
            [remaining_seats[0]],
            prefs,
            debug)
        pick_this = optimal_seating(
            assigned_seats + [remaining_seats[0]],
            [remaining_seats[1]],
            prefs,
            debug)
    else:
        pick_next = optimal_seating(
            assigned_seats + [remaining_seats[1]],
            [remaining_seats[0]] + remaining_seats[2:],
            prefs,
            debug)
        pick_this = optimal_seating(
            assigned_seats + [remaining_seats[0]],
            remaining_seats[1:],
            prefs,
            debug)

    if pick_next[1] > pick_this[1]:
        return pick_next
    else:
        return pick_this

In [14]:
optimal_seating([], list(prefs.keys()), prefs, True)

assigned_seats=[], remaining_seats=['Alice', 'Bob', 'Carol', 'David']
assigned_seats=['Bob'], remaining_seats=['Alice', 'Carol', 'David']
assigned_seats=['Bob', 'Carol'], remaining_seats=['Alice', 'David']
assigned_seats=['Bob', 'Carol', 'David'], remaining_seats=['Alice']
assigned_seats_n=['Bob', 'Carol', 'David', 'Alice'] evaluate to happiness=330
assigned_seats=['Bob', 'Carol', 'Alice'], remaining_seats=['David']
assigned_seats_n=['Bob', 'Carol', 'Alice', 'David'] evaluate to happiness=-114
assigned_seats=['Bob', 'Alice'], remaining_seats=['Carol', 'David']
assigned_seats=['Bob', 'Alice', 'David'], remaining_seats=['Carol']
assigned_seats_n=['Bob', 'Alice', 'David', 'Carol'] evaluate to happiness=330
assigned_seats=['Bob', 'Alice', 'Carol'], remaining_seats=['David']
assigned_seats_n=['Bob', 'Alice', 'Carol', 'David'] evaluate to happiness=22
assigned_seats=['Alice'], remaining_seats=['Bob', 'Carol', 'David']
assigned_seats=['Alice', 'Carol'], remaining_seats=['Bob', 'David']
assign

(['Alice', 'Bob', 'Carol', 'David'], 330)

In [15]:
def my_part1_solution(data, debug=False):
    prefs = pivot_preferences(
        parse_data(
            data))
    return optimal_seating(
        [],
        list(prefs.keys()),
        prefs,
        debug)

In [16]:
seating, happiness = my_part1_solution(testdata, True)
assert happiness == 330
print(seating)

assigned_seats=[], remaining_seats=['Alice', 'Bob', 'Carol', 'David']
assigned_seats=['Bob'], remaining_seats=['Alice', 'Carol', 'David']
assigned_seats=['Bob', 'Carol'], remaining_seats=['Alice', 'David']
assigned_seats=['Bob', 'Carol', 'David'], remaining_seats=['Alice']
assigned_seats_n=['Bob', 'Carol', 'David', 'Alice'] evaluate to happiness=330
assigned_seats=['Bob', 'Carol', 'Alice'], remaining_seats=['David']
assigned_seats_n=['Bob', 'Carol', 'Alice', 'David'] evaluate to happiness=-114
assigned_seats=['Bob', 'Alice'], remaining_seats=['Carol', 'David']
assigned_seats=['Bob', 'Alice', 'David'], remaining_seats=['Carol']
assigned_seats_n=['Bob', 'Alice', 'David', 'Carol'] evaluate to happiness=330
assigned_seats=['Bob', 'Alice', 'Carol'], remaining_seats=['David']
assigned_seats_n=['Bob', 'Alice', 'Carol', 'David'] evaluate to happiness=22
assigned_seats=['Alice'], remaining_seats=['Bob', 'Carol', 'David']
assigned_seats=['Alice', 'Carol'], remaining_seats=['Bob', 'David']
assign

In [17]:
my_part1_solution(inputdata, False)

(['Alice', 'Bob', 'David', 'Eric', 'Carol', 'Frank', 'George', 'Mallory'], 483)

Ok... `483` is indeed the highest happiness I find, but there supposedly exist a better solution.

In [18]:
from itertools import permutations
from math import inf


def my_part1_solution_itertools(data, debug=False):
    prefs = pivot_preferences(
        parse_data(
            data))
    seating_best = []
    happiness_best = -inf
    for seating in permutations(list(prefs.keys())):
        happiness_this = evaluate_seating(seating, prefs)
        if happiness_this > happiness_best:
            seating_best = seating
            happiness_best = happiness_this
    return seating_best, happiness_best

In [19]:
seating, happiness = my_part1_solution_itertools(testdata, True)
assert happiness == 330
print(seating)

('Alice', 'Bob', 'Carol', 'David')


In [20]:
seating, happiness = my_part1_solution_itertools(inputdata, True)
print(f'{seating=}, {happiness=}')

seating=('Alice', 'Bob', 'Frank', 'Carol', 'Eric', 'David', 'George', 'Mallory'), happiness=733


In [21]:
HTML(downloaded['part1_footer'])

## Part Two

In [22]:
HTML(downloaded['part2'])

In [23]:
def evaluate_seating_including_me(seating, prefs):
    happiness = 0
    for i, s in enumerate(seating):
        if s == 'Me':
            continue

        if i == 0:
            right = seating[-1]
        else:
            right = seating[i - 1]

        if i == len(seating) - 1:
            left = seating[0]
        else:
            left = seating[i + 1]

        for n in left, right:
            if n != 'Me':
                happiness += prefs[s][n]
    return happiness


def my_part2_solution(data, debug=False):
    prefs = pivot_preferences(
        parse_data(
            data))
    seating_best = []
    happiness_best = -inf
    for seating in permutations(['Me'] + list(prefs.keys())):
        happiness_this = evaluate_seating_including_me(seating, prefs)
        if happiness_this > happiness_best:
            if debug:
                print(f'{happiness_this=}')
            seating_best = seating
            happiness_best = happiness_this
    return seating_best, happiness_best

In [24]:
seating_w_me, happiness_w_me = my_part2_solution(inputdata, False)
print(f'{seating_w_me=}, {happiness_w_me=}')
print(f'{abs(happiness_w_me - happiness)}')

seating_w_me=('Me', 'George', 'David', 'Eric', 'Carol', 'Frank', 'Bob', 'Alice', 'Mallory'), happiness_w_me=725
8


In [25]:
HTML(downloaded['part2_footer'])

Meh, that's so frustrating, a poorly phrased question?!